In [1]:
from deep_orderbook.shaper import BookShaper, ArrayShaper
import numpy as np
from pylab import rcParams

In [2]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [3]:
MARKETS = ["ETH-BTC", "BTC-USD", "ETH-USD"]

In [4]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [ ]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed
from deep_orderbook.replayer import ParquetReplayer

async def iter_sec(date_regexp='2024-08-06', max_samples=0):
    replayer = ParquetReplayer('data', date_regexp=date_regexp)
    async with CoinbaseFeed(
        markets=MARKETS,
        replayer=replayer,
    ) as feed:
        async for onesec in feed.one_second_iterator():
            yield onesec
            if max_samples == 0:
                break
            else:
                max_samples -= 1

async for onesec in iter_sec(max_samples=3):
    print(onesec)

In [ ]:
async def iter_shapes(date_regexp='2024-08-06', max_samples=0):
    symbol_shapers = {pair: ArrayShaper() for pair in MARKETS}
    async for onesec in iter_sec(max_samples=2):
        yield {
            market: await symbol_shapers[market].make_arr3d(
                onesec.symbols[market]
            )
            for market in MARKETS
        }
        if max_samples == 0:
            break
        else:
            max_samples -= 1


async for shaped in iter_shapes(max_samples=3):
    print(shaped)


In [ ]:
multi_replay = file_replayer.multireplayL2_async(MARKETS)
d = await next(multi_replay)
d

In [ ]:
async def run(maxpoints=10):
    async for i in iter(range(maxpoints)):
        d = await next(multi_replay)
    #return d

task = asyncio.create_task(run())
await task

In [ ]:
async def iter_gen_arrays(date_regexp='2024-08-06', max_samples=0):
    shaper = BookShaper()
    async for onesec in iter_sec(max_samples=max_samples):
        yield await shaper.gen_array_async(onesec)

async for arr in iter_gen_arrays(max_samples=2):
    print(arr)

genarr = shaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await next(genarr)
_

In [ ]:
genacc = shaper.accumulate_array(genarr, markets=MARKETS)
_ = await next(genacc)
_

In [ ]:
async for toshow in BookShaper.images(accumulated_arrays=genacc, every=1, LENGTH=512):
    plt.figure(figsize=(15,11))
    plt.imshow(toshow, origin="upper")
    clear_output(wait=True)
    plt.show()

In [ ]:
every = 2
LENGTH = 256
x = []
async for n,sec in enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower")
        plt.show()
        # pr = np.stack(data['ps'])[-LENGTH:, :-1]
        # plt.plot(pr)
        # plt.show()
#        print(symb)
        clear_output(wait=True)
#        plt.show()
#    if n == 100:
#        break
    pass